# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [63]:
import logging
import os
import io
import joblib
import json
import math
import requests
import numpy as np
import pandas as pd
import datetime
import azureml.core
import azureml.automl

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.42.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'automl-forecast'

experiment=Experiment(ws, experiment_name)

quick-starts-ws-200358
aml-quickstarts-200358
southcentralus
81cefad3-d2c9-4f77-a466-99a7f541c7bb


In [3]:
# Create or attach a compute cluster
cluster_name = "canina-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=10)

A new compute target has been created.
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
DATA_DIR = os.path.join(os.path.abspath(os.getcwd()), "ojdata")
LAST_WEEK = 138
NUM_WEEK_TEST = 3
time_column_name = "week_start"
grain_column_names = ["store", "brand"]
target = "move"

# The start datetime of the first week in the record
FIRST_WEEK_START = pd.to_datetime("1989-09-14 00:00:00")

df = pd.read_csv(os.path.join(DATA_DIR, "yx.csv"))
df = df.loc[df.week <= LAST_WEEK]

In [5]:
# Convert logarithm of the unit sales to unit sales
df["move"] = df["logmove"].apply(lambda x: round(math.exp(x)))

# Add timestamp column
df["week_start"] = df["week"].apply(lambda x: FIRST_WEEK_START + datetime.timedelta(days=(x - 1) * 7))

In [6]:
# Split data into training and test sets

def split_last_n_by_grain(df, n):
    """Group df by grain and split on last n rows for each group."""
    df_grouped = df.sort_values(time_column_name).groupby(
        grain_column_names, group_keys=False
    )
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    return df_head, df_tail

# Drop "logmove", because it is a leaky feature
df.drop("logmove", axis=1, inplace=True)

train_df, test_df = split_last_n_by_grain(df, NUM_WEEK_TEST)

train_df.reset_index(drop=True)
test_df.reset_index(drop=True)

local_data_paths = [
    os.path.join(DATA_DIR, "train_automl.csv"),
    os.path.join(DATA_DIR, "test_automl.csv")
]

train_df.to_csv(local_data_paths[0], index=None, header=True)
test_df.to_csv(local_data_paths[1], index=None, header=True)

In [7]:
# Upload data to datastore

ds = ws.get_default_datastore()
ds.upload_files(
    files=local_data_paths, 
    target_path="dataset/", 
    overwrite=True, 
    show_progress=False)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


$AZUREML_DATAREFERENCE_70c27ba3602842e19838892db7dd042e

In [8]:
train_dataset = Dataset.Tabular.from_delimited_files(path=ds.path("dataset/train_automl.csv"))

In [9]:
# Visualize the first five lines of the training dataset

train_dataset.to_pandas_dataframe().head(5)

,store,brand,week,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,move,week_start
0,2,1,40,1,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1,0.0,37.992326,8256,1990-06-14
1,2,1,46,1,0.060469,0.060312,0.045156,0.046719,0.049531,0.047813,0.045781,0.027969,0.042969,0.042031,0.038984,0,0.0,30.126667,6144,1990-07-26
2,2,1,47,1,0.060469,0.060312,0.045156,0.046719,0.037344,0.053021,0.045781,0.041406,0.048125,0.032656,0.038984,0,0.0,30.000000,3840,1990-08-02
3,2,1,48,1,0.060469,0.060312,0.049844,0.037344,0.049531,0.053021,0.045781,0.041406,0.042344,0.032656,0.038984,0,0.0,29.950000,8000,1990-08-09
4,2,1,50,1,0.060469,0.060312,0.043594,0.031094,0.049531,0.053021,0.046648,0.041406,0.042344,0.032656,0.038203,0,0.0,29.920000,8896,1990-08-23


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# Forecast settings
forecast_settings = {
    "time_column_name": time_column_name,
    "grain_column_names": grain_column_names,
    "max_horizon": NUM_WEEK_TEST
}
# Automl settings 
automl_settings = {
    "experiment_timeout_hours" : 1.2,
    "primary_metric" : "mean_absolute_percentage_error",
    "n_cross_validations" : 3
}

# Automl config 
automl_config = AutoMLConfig(
    task="forecasting",
    debug_log="automl_errors.log",
    training_data=train_dataset,
    label_column_name=target,
    compute_target=compute_target,
    enable_early_stopping=True,
    verbosity=logging.INFO,
    **automl_settings,
    **forecast_settings
)

In [11]:
# Submit the experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-forecast,AutoML_c2e81835-efbf-42ad-9fe5-6daf5e505a99,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [41]:
# Provide details about the performance of the different models on the primary metric "normalized mean absolute error"
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-forecast,AutoML_c2e81835-efbf-42ad-9fe5-6daf5e505a99,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Time Series ID detection
STATUS:       PASSED
DESCRIPTION:  The data set was analyzed, and no duplicate time index were detected.
              Learn more about time-series forecasting configurations: https://aka.ms/AutomatedMLForecastingConfiguration

********************************************************************************************

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed, all data points are aligned with detected frequency.
              

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************

{'runId': 'AutoML_c2e81835-efbf-42ad-9fe5-6daf5e505a99',
 'target': 'canina-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-07-06T06:52:36.799958Z',
 'endTimeUtc': '2022-07-06T08:25:18.246592Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 1 hour(s) 12 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'canina-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-forecast","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-200358","workspace_name":"quick-starts-ws-200358","region":"southcentralus","compute_target":"canina-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pip

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
# Get the best model
best_run_automl, best_automl_model = remote_run.get_output()

In [15]:
# List all files of the best run
# We will use it to download the model, the environment file and the scoring file

best_run_automl.get_file_names()

['automl_driver.py',
 'explanation/18ac9f8c/eval_data_viz.interpret.json',
 'explanation/18ac9f8c/expected_values.interpret.json',
 'explanation/18ac9f8c/features.interpret.json',
 'explanation/18ac9f8c/global_names/0.interpret.json',
 'explanation/18ac9f8c/global_rank/0.interpret.json',
 'explanation/18ac9f8c/global_values/0.interpret.json',
 'explanation/18ac9f8c/local_importance_values.interpret.json',
 'explanation/18ac9f8c/rich_metadata.interpret.json',
 'explanation/18ac9f8c/visualization_dict.interpret.json',
 'explanation/18ac9f8c/ys_pred_viz.interpret.json',
 'explanation/281a9c20/expected_values.interpret.json',
 'explanation/281a9c20/features.interpret.json',
 'explanation/281a9c20/global_names/0.interpret.json',
 'explanation/281a9c20/global_rank/0.interpret.json',
 'explanation/281a9c20/global_values/0.interpret.json',
 'explanation/281a9c20/local_importance_values.interpret.json',
 'explanation/281a9c20/rich_metadata.interpret.json',
 'explanation/281a9c20/visualization_d

In [17]:
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

In [18]:
# Download the best AutoML model
joblib.dump(value = best_automl_model,
           filename = 'model.pkl')

['model.pkl']

In [19]:
joblib.load('model.pkl')

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(country_or_region=None, drop_column_names=[], featurization_config=FeaturizationConfig(blocked_transformers=None, column_purposes=None, dataset_language=None, prediction_transform_type=None, transformer_params=None), force_time_index_features=Non...
                                                     PreFittedSoftVotingRegressor(estimators=[('2', Pipeline(memory=None, steps=[('average', Average(timeseries_param_dict={'time_column_name': 'week_start', 'grain_column_names': ['store', 'brand'], 'drop_column_names': [], 'overwrite_columns': True, 'dropna': False, 'transform_dictionary': {'min': '_automl_target_col', 'max': '_automl_target_col', 'mean': '_automl_target_col'}, 'max_horizon': 3, 'origin_time_colname': 'origin', 'country_or_region': None, 'n_cross_validations': 3, 'sho

In [21]:
run_metrics = best_run_automl.get_metrics()
for m, val in run_metrics.items():
    print(f"{m}: {val}")

median_absolute_error: 2263.019157560532
normalized_median_absolute_error: 0.07499737912954695
root_mean_squared_error: 31927.000301509164
r2_score: 0.07548594845059264
root_mean_squared_log_error: 0.8695950502106419
normalized_root_mean_squared_error: 0.14071916098297987
explained_variance: 0.08800651717195869
mean_absolute_percentage_error: 95.58172663345624
spearman_correlation: 0.7565418928644784
mean_absolute_error: 9682.832564720467
normalized_root_mean_squared_log_error: 0.1945550504526623
normalized_mean_absolute_error: 0.10960286633862626
residuals: aml://artifactId/ExperimentRun/dcid.AutoML_c2e81835-efbf-42ad-9fe5-6daf5e505a99_20/residuals
predicted_true: aml://artifactId/ExperimentRun/dcid.AutoML_c2e81835-efbf-42ad-9fe5-6daf5e505a99_20/predicted_true
forecast_table: aml://artifactId/ExperimentRun/dcid.AutoML_c2e81835-efbf-42ad-9fe5-6daf5e505a99_20/forecast_table


In [22]:
best_run_automl.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":1}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 '_aml_system_codegen': 'completed',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_c2e81835-efbf-42ad-9fe5-6daf5e505a99_ModelExplain',
 'model_explanation': 'True'}

In [23]:
print(best_automl_model.steps)

[('timeseriestransformer', TimeSeriesTransformer(country_or_region=None, drop_column_names=[], featurization_config=FeaturizationConfig(blocked_transformers=None, column_purposes=None, dataset_language=None, prediction_transform_type=None, transformer_params=None), force_time_index_features=None, freq='W-THU', grain_column_names=['store', 'brand'], group=None, lookback_features_removed=False, max_horizon=3, origin_time_colname='origin', pipeline=Pipeline(memory=None, steps=[('make_numeric_na_dummies', MissingDummiesTransformer(numerical_columns=['week', 'constant', 'price1', 'price2', 'price3', 'price4', 'price5', 'price6', 'price7', 'price8', 'price9', 'price10', 'price11', 'deal', 'feat', 'profit'])), ('impute_na_numeric_datetime', TimeSeriesImputer(end=None, freq='W-THU', impute_by_horizon=False, input_column=['week', 'constant', 'price1', 'price2', 'price3', 'price4', 'price5', 'price6', 'price7', 'price8', 'price9', 'price10', 'price11', 'deal', 'feat', 'profit'], limit=None, limi

In [24]:
best_run_automl.download_file('outputs/featurization_summary.json', 'outputs/featurization_summary.json')

In [25]:
with open('outputs/featurization_summary.json', 'r') as features:
    features_summary = json.load(features)
    
df_features = pd.DataFrame.from_records(features_summary)

In [26]:
df_features

,RawFeatureName,TypeDetected,Dropped,EngineeredFeatureCount,Transformations,TransformationParams
0,week,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['week'], 'Transfor..."
1,constant,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['constant'], 'Tran..."
2,price1,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price1'], 'Transf..."
3,price2,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price2'], 'Transf..."
4,price3,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price3'], 'Transf..."
5,price4,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price4'], 'Transf..."
6,price5,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price5'], 'Transf..."
7,price6,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price6'], 'Transf..."
8,price7,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price7'], 'Transf..."
9,price8,Numeric,No,2,"[MedianImputer, ImputationMarker]","{'Transformer1': {'Input': ['price8'], 'Transf..."


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [27]:
# Register the best AutoML model
automl_model_name = best_run_automl.properties["model_name"]
model_reg = remote_run.register_model(model_name = automl_model_name,
                                      description = "AutoML model for forecasting.",
                                      tags = None)

In [28]:
model_reg

Model(workspace=Workspace.create(name='quick-starts-ws-200358', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-200358'), name=AutoMLc2e81835e20, id=AutoMLc2e81835e20:1, version=1, tags={}, properties={})

In [29]:
model_reg.download(target_dir='outputs', exist_ok=True)

'outputs/model.pkl'

In [30]:
print("Model ID", remote_run.model_id)

Model ID AutoMLc2e81835e20


In [31]:
# Downoad the file containing the environment details 
best_run_automl.download_file('outputs/conda_env_v_1_0_0.yml', 'runenv.yml')

# Download the scoring file
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [32]:
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'runenv.yml')

In [33]:
inference_config = InferenceConfig(entry_script = 'score_automl.py',
                                   environment = myenv)

In [34]:
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=4,
    description="AutoML model to forecast Orange Juice data"
)

In [42]:
service = Model.deploy(
    workspace=ws,
    name="automl-forecast-oj",
    models=[model_reg],
    inference_config=inference_config,
    deployment_config=aciconfig
)

In [43]:
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-06 09:51:00+00:00 Creating Container Registry if not exists..
2022-07-06 10:01:01+00:00 Registering the environment.
2022-07-06 10:01:02+00:00 Building image..
2022-07-06 10:21:19+00:00 Generating deployment configuration.
2022-07-06 10:21:20+00:00 Submitting deployment to compute.
2022-07-06 10:21:23+00:00 Checking the status of deployment automl-forecast-oj..
2022-07-06 10:23:56+00:00 Checking the status of inference endpoint automl-forecast-oj.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [44]:
print(service.state)

Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [45]:
test_df = pd.read_csv("ojdata/test_automl.csv")

In [46]:
# We will skip the week "136" to leave time for planning inventory as in real life.
# We do our forecast on the two weeks "137" and "138"

test_df.drop(test_df[test_df["week"] <= 136].index, inplace=True)

In [47]:
# Prepare data to test it on the web service
X = test_df
ground_truth = test_df.pop(target).values

In [48]:
# Convert DateTime to String, because Object of type Timestamp is not JSON serializable
X[time_column_name] = X[time_column_name].astype(str)

In [49]:
X.shape

(1826, 19)

In [50]:
X.head()

,store,brand,week,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,week_start
1,2,1,137,1,0.04,0.05,0.05,0.04,0.03,0.04,0.03,0.04,0.04,0.02,0.03,0,0.00,20.43,1992-04-23
2,2,1,138,1,0.04,0.04,0.05,0.04,0.04,0.05,0.04,0.04,0.04,0.03,0.03,1,1.00,11.29,1992-04-30
4,2,2,137,1,0.04,0.05,0.05,0.04,0.03,0.04,0.03,0.04,0.04,0.02,0.03,0,0.00,33.30,1992-04-23
5,2,2,138,1,0.04,0.04,0.05,0.04,0.04,0.05,0.04,0.04,0.04,0.03,0.03,1,1.00,9.43,1992-04-30
7,2,3,137,1,0.04,0.05,0.05,0.04,0.03,0.04,0.03,0.04,0.04,0.02,0.03,0,0.00,30.51,1992-04-23


In [71]:
# Take NB_SAMPLES samples from all the test dataset
START_SAMPLES = 0
END_SAMPLES = 1000
test_X = X.iloc[START_SAMPLES:END_SAMPLES,:]

In [72]:
test_X.shape

(1000, 19)

In [73]:
req_data = json.dumps({"data": test_X.to_dict(orient='records')})

In [74]:
headers = {'Content-Type':'application/json'}

In [75]:
response = requests.post(service.scoring_uri, req_data, headers=headers)

In [76]:
print(response.text)

"{\"forecast\": [12808.533333333335, 12808.533333333335, 8671.2, 8671.2, 2987.7333333333336, 2987.7333333333336, 13870.933333333336, 13870.933333333336, 12825.600000000002, 12825.600000000002, 2832.8, 2832.8, 3776.5333333333338, 3776.5333333333338, 1855.4666666666667, 1855.4666666666667, 3322.666666666667, 3322.666666666667, 13689.066666666668, 13689.066666666668, 6816.0, 6816.0, 13347.2, 13347.2, 9028.0, 9028.0, 4757.333333333334, 4757.333333333334, 12884.266666666666, 12884.266666666666, 16314.666666666666, 16314.666666666666, 4947.200000000001, 4947.200000000001, 5155.2, 5155.2, 2697.6, 2697.6, 3768.5333333333338, 3768.5333333333338, 14614.933333333334, 14614.933333333334, 11715.2, 11715.2, 15621.333333333334, 15621.333333333334, 7233.6, 7233.6, 2829.3333333333335, 2829.3333333333335, 24461.866666666665, 24461.866666666665, 26051.2, 26051.2, 5484.8, 5484.8, 6115.7333333333345, 6115.7333333333345, 6449.6, 6449.6, 5848.533333333335, 5848.533333333335, 28658.13333333334, 28658.13333333

In [77]:
result = json.loads(json.loads(response.text))

In [96]:
result["index"][0]

{'week_start': 703987200000, 'store': 2, 'brand': 1}

In [81]:
gd_truth_test = ground_truth[START_SAMPLES:END_SAMPLES]

df_result = pd.DataFrame(result["index"])
df_result[time_column_name] = pd.to_datetime(df_result[time_column_name], unit="ms")
df_result["ground_truth"] = gd_truth_test
df_result["forecast"] = result["forecast"]
df_result["prediction_interval"] = result["prediction_interval"]

In [82]:
test_size = len(test_X)

In [83]:
df_result.head(test_size)

,week_start,store,brand,ground_truth,forecast,prediction_interval
0,1992-04-23,2,1,9792,12808.53,"[-75091.55491913024, 100708.62158579691]"
1,1992-04-30,2,1,16960,12808.53,"[-94846.64894873949, 120463.71561540614]"
2,1992-04-23,2,2,6240,8671.20,"[-79228.88825246358, 96571.28825246356]"
3,1992-04-30,2,2,14784,8671.20,"[-98983.98228207283, 116326.3822820728]"
4,1992-04-23,2,3,1920,2987.73,"[-84912.35491913024, 90887.8215857969]"
...,...,...,...,...,...,...
995,1992-04-30,91,3,448,2105.60,"[-105549.58228207282, 109760.7822820728]"
996,1992-04-23,91,4,2880,13400.53,"[-74499.55491913024, 101300.62158579691]"
997,1992-04-30,91,4,11968,13400.53,"[-94254.64894873949, 121055.71561540614]"
998,1992-04-23,91,5,27840,13946.13,"[-73953.95491913025, 101846.2215857969]"


In [92]:
# Compute MAPE metric of the test data

def compute_mape(ground_truth, forecasts) -> float:
    ground_truth, forecasts = np.array(ground_truth), np.array(forecasts)
    mape = np.mean(np.abs((ground_truth-forecasts)/ground_truth))*100
    return round(mape,2)

In [93]:
forecasts = df_result["forecast"].values
print("MAPE : ",compute_mape(gd_truth_test, forecasts),"%")

MAPE :  137 %


TODO: In the cell below, print the logs of the web service and delete the service

In [97]:
# Enable application insights
service.update(enable_app_insights=True)

In [98]:
# Print the logs of the Web service
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2022-07-06T11:11:06,600800491+00:00 - iot-server/run 
2022-07-06T11:11:06,603129291+00:00 - gunicorn/run 
2022-07-06T11:11:06,603198791+00:00 - rsyslog/run 
2022-07-06T11:11:06,607883491+00:00 | gunicorn/run | 
2022-07-06T11:11:06,612178991+00:00 | gunicorn/run | ###############################################
2022-07-06T11:11:06,614388991+00:00 | gunicorn/run | AzureML Container Runtime Information
bash: /azureml-envs/azureml_7

 # DELETE

In [99]:
# Remove the web service
service.delete()

# Remove the compute cluster
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"



**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
